In [5]:
#Import Hamilton County voter data file pulled from https://www6.sos.state.oh.us/ords/f?p=111:1
import pandas as pd
hc_full = pd.read_csv('../HAMILTON.txt')

C:\Users\Eliza\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (58,64,66,67,73,74,78,79,82,87,93,96,97,100,103) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
geocoding_columns = ['SOS_VOTERID', 'RESIDENTIAL_ADDRESS1',  'RESIDENTIAL_CITY',
 'RESIDENTIAL_STATE', 'RESIDENTIAL_ZIP']

geocode_subset = hc_full[geocoding_columns]

In [7]:
#Count the number of NaN entries for each column & percent of total
def missing_table(df):
        mis_val = df.isna().sum()
        mis_val_percent = 100 * df.isna().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

mt = missing_table(geocode_subset)
mt

Your dataframe has 5 columns and 574834 Rows.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,Data Type


In [9]:
geocode_subset.head()

,SOS_VOTERID,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP
0,OH0013910067,1672 DEVILS BACKBONE RD,CINCINNATI,OH,45233
1,OH0013638654,1448 FIELDCREST CT,CINCINNATI,OH,45231
2,OH0013744969,7206 MIAMI HILLS DR,CINCINNATI,OH,45243
3,OH0020648383,11606 HANOVER RD,CINCINNATI,OH,45240
4,OH0021001272,7822 MITCHELL PARK DR,CLEVES,OH,45002


In [11]:
#A little data exploration
geocode_subset.groupby(['RESIDENTIAL_CITY']).size()

RESIDENTIAL_CITY
ADDYSTON            527
CAMP DENNISON       293
CINCINNATI       534163
CLEVES             9649
HAMILTON             26
HARRISON          13614
HOOVEN              188
LOVELAND          11142
MIAMITOWN            76
MILFORD              13
MT ST JOSEPH        161
NORTH BEND         3163
TERRACE PARK       1819
dtype: int64

In [12]:
geocode_subset.groupby(['RESIDENTIAL_STATE']).size()

RESIDENTIAL_STATE
OH    574834
dtype: int64

In [15]:
#save some time by only geocoding unique addresses - since multiple people can live at the same address
gc_unique = geocode_subset.drop(columns='SOS_VOTERID').drop_duplicates()
gc_unique.shape[0]

238280

In [23]:
#create a new index for each unique address
gc = gc_unique.reset_index().drop(columns='index')
gc.head()

,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP
0,1672 DEVILS BACKBONE RD,CINCINNATI,OH,45233
1,1448 FIELDCREST CT,CINCINNATI,OH,45231
2,7206 MIAMI HILLS DR,CINCINNATI,OH,45243
3,11606 HANOVER RD,CINCINNATI,OH,45240
4,7822 MITCHELL PARK DR,CLEVES,OH,45002


In [27]:
#test batch of 2000
test = gc.iloc[0:1999]
test.to_csv('../test.csv', header=False)

In [21]:
#Test api call & return
import requests

input_file = '../test.csv'
output_file = '../codedtest.csv'

url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
files = {'file': open(input_file, 'rb')}
keywords = {'benchmark': 'Public_AR_Census2010', 'vintage': 'Census2010_Census2010'}
#layers : 14
#format: json
r = requests.get(url, data=keywords, files=files)

In [22]:
r.text

'Parameter conditions &quot;form&quot; not met for actual request parameters: '

In [19]:
#I am going to create a column that contains all address info needed by API
gc_unique['API_STRING']='street='+gc_unique['RESIDENTIAL_ADDRESS1']+'&city='+gc_unique['RESIDENTIAL_CITY']+'&state='+gc_unique['RESIDENTIAL_STATE']+'&zip='+gc_unique['RESIDENTIAL_ZIP'].astype(str)
gc_unique['API_STRING'].head()

0    street=1672 DEVILS BACKBONE RD&city=CINCINNATI...
1    street=1448 FIELDCREST CT&city=CINCINNATI&stat...
2    street=7206 MIAMI HILLS DR&city=CINCINNATI&sta...
3    street=11606 HANOVER RD&city=CINCINNATI&state=...
4    street=7822 MITCHELL PARK DR&city=CLEVES&state...
Name: API_STRING, dtype: object

In [21]:
gc_unique['API_NOSPACE']=gc_unique['API_STRING'].str.replace(" ", "+")
gc_unique['API_NOSPACE'].head()

0    street=1672+DEVILS+BACKBONE+RD&city=CINCINNATI...
1    street=1448+FIELDCREST+CT&city=CINCINNATI&stat...
2    street=7206+MIAMI+HILLS+DR&city=CINCINNATI&sta...
3    street=11606+HANOVER+RD&city=CINCINNATI&state=...
4    street=7822+MITCHELL+PARK+DR&city=CLEVES&state...
Name: API_NOSPACE, dtype: object

In [24]:
#Create address list & a test string
a_list = gc_unique['API_NOSPACE'].to_list()
test_string = a_list[0]
test_string

'street=1672+DEVILS+BACKBONE+RD&city=CINCINNATI&state=OH&zip=45233'

In [50]:
#Test api call & return
#reference - https://towardsdatascience.com/creating-a-dataset-using-an-api-with-python-dcc1607616d
import numpy as np
import requests
import json
url = "https://geocoding.geo.census.gov/geocoder/geographies/address?"+test_string+"&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
JSONContent = requests.get(url).json()
content = json.dumps(JSONContent, indent = 4, sort_keys=True)
print(content)

{
    "result": {
        "addressMatches": [
            {
                "addressComponents": {
                    "city": "CINCINNATI",
                    "fromAddress": "1666",
                    "preDirection": "",
                    "preQualifier": "",
                    "preType": "",
                    "state": "OH",
                    "streetName": "Devils Backbone",
                    "suffixDirection": "",
                    "suffixQualifier": "",
                    "suffixType": "Rd",
                    "toAddress": "1698",
                    "zip": "45233"
                },
                "coordinates": {
                    "x": -84.65108,
                    "y": 39.125805
                },
                "geographies": {
                    "Census Blocks": [
                        {
                            "AREALAND": 494474,
                            "AREAWATER": 0,
                            "BASENAME": "2003",
                            "BL

In [75]:
JSONContent['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['BLOCK']

'2003'

In [86]:
#I want to try a batch of 100 just to check things out before I have this run for a while

a_list100 = a_list[:99]
results_list = []

for i in a_list100:
    url = "https://geocoding.geo.census.gov/geocoder/geographies/address?"+i+"&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
    JSONContent = requests.get(url).json()
    if 'error' not in JSONContent:
        match = JSONContent['result']['addressMatches']
        for a in match:
            x = a['coordinates']['x']
            y = a['coordinates']['y']
            matchedAddress = a['matchedAddress']
            census = a['geographies']['Census Blocks']
            for c in census:
                block = c['BLOCK']
                tract = c['TRACT']
                results_list.append([x, y, matchedAddress, block, tract])
dataset = pd.DataFrame(results_list)

In [87]:
dataset
#so some of these address are obviously not matching - I need to store these somehow
#I also want to make the code above into a function

,0,1,2,3,4
0,-84.651080,39.125805,"1672 Devils Backbone Rd, CINCINNATI, OH, 45233",2003,021201
1,-84.369520,39.195637,"7206 Miami Hills Dr, CINCINNATI, OH, 45243",4005,024100
2,-84.511220,39.290646,"11606 Hanover Rd, CINCINNATI, OH, 45240",3006,021506
3,-84.700990,39.177185,"7822 Mitchell Park Dr, CLEVES, OH, 45002",3002,020403
4,-84.797714,39.249740,"447 Lyness Ave, HARRISON, OH, 45030",4003,026101
...,...,...,...,...,...
88,-84.316900,39.086082,"8521 Holidayhills Dr, CINCINNATI, OH, 45255",1002,025102
89,-84.616680,39.138218,"3416 Lumardo Ave, CINCINNATI, OH, 45238",3009,010201
90,-84.564920,39.105946,"3316 W 8th St, CINCINNATI, OH, 45205",3005,009300
91,-84.359055,39.232410,"9701 Zig Zag Rd, CINCINNATI, OH, 45242",4002,023902


In [96]:
def geocoder(clist):
    results_list=[]
    errors_list=[]
    for i in clist:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/address?"+i+"&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
        JSONContent = requests.get(url).json()
        if 'error' not in JSONContent:
            match = JSONContent['result']['addressMatches']
            for a in match:
                x = a['coordinates']['x']
                y = a['coordinates']['y']
                matchedAddress = a['matchedAddress']
                census = a['geographies']['Census Blocks']
                for c in census:
                    block = c['BLOCK']
                    tract = c['TRACT']
                    results_list.append([x, y, matchedAddress, block, tract])
        else:
            errors_list.append([i])
    output = pd.DataFrame(results_list)
    output_errors = pd.DataFrame(errors_list)
    return output, output_errors

In [97]:
a_list100 = a_list[:99]
out1, error1 = geocoder(a_list100)

In [98]:
out1

,0,1,2,3,4
0,-84.651080,39.125805,"1672 Devils Backbone Rd, CINCINNATI, OH, 45233",2003,021201
1,-84.369520,39.195637,"7206 Miami Hills Dr, CINCINNATI, OH, 45243",4005,024100
2,-84.511220,39.290646,"11606 Hanover Rd, CINCINNATI, OH, 45240",3006,021506
3,-84.700990,39.177185,"7822 Mitchell Park Dr, CLEVES, OH, 45002",3002,020403
4,-84.797714,39.249740,"447 Lyness Ave, HARRISON, OH, 45030",4003,026101
...,...,...,...,...,...
88,-84.316900,39.086082,"8521 Holidayhills Dr, CINCINNATI, OH, 45255",1002,025102
89,-84.616680,39.138218,"3416 Lumardo Ave, CINCINNATI, OH, 45238",3009,010201
90,-84.564920,39.105946,"3316 W 8th St, CINCINNATI, OH, 45205",3005,009300
91,-84.359055,39.232410,"9701 Zig Zag Rd, CINCINNATI, OH, 45242",4002,023902


In [99]:
error1
#so error catching is sitll a work in progress, but I can run this and match what I can
#then generate a list of everything that didn't code at the end

""


In [100]:
len(a_list)
#my current function takes forever to run, so I'm going to work from a small sample for proof of concept

238280

In [123]:
geocoding_columns = ['SOS_VOTERID', 'RESIDENTIAL_ADDRESS1', 'RESIDENTIAL_CITY',
 'RESIDENTIAL_STATE', 'RESIDENTIAL_ZIP', 'PRIMARY-03/15/2016', 'GENERAL-11/08/2016']

geocode_subset = hc_full[geocoding_columns]
gc_working=geocode_subset[geocode_subset['PRIMARY-03/15/2016'].notna()]
gc_working=gc_working[gc_working['GENERAL-11/08/2016'].isna()]
gc_sample = gc_working.sample(n=500, random_state=42)
gc_sample.head()

,SOS_VOTERID,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,PRIMARY-03/15/2016,GENERAL-11/08/2016
307511,OH0024178584,3228 BERWYN PL,CINCINNATI,OH,45209,R,NaN
85535,OH0013709625,1717 FERNWOOD ST,CINCINNATI,OH,45206,R,NaN
502348,OH0024705077,3731 WOODLAND AVE,CINCINNATI,OH,45209,R,NaN
125756,OH0013738913,838 OAK CANYON DR,LOVELAND,OH,45140,R,NaN
66682,OH0022516459,3018 KINMONT ST,CINCINNATI,OH,45208,R,NaN


In [127]:
#copy paste code above
gc_sample['API_STRING']='street='+gc_sample['RESIDENTIAL_ADDRESS1']+'&city='+gc_sample['RESIDENTIAL_CITY']+'&state='+gc_sample['RESIDENTIAL_STATE']+'&zip='+gc_sample['RESIDENTIAL_ZIP'].astype(str)
gc_sample['API_NOSPACE']=gc_sample['API_STRING'].str.replace(" ", "+")

In [128]:
a_list = gc_sample['API_NOSPACE'].to_list()
len(a_list)

500

In [129]:
out1, error1 = geocoder(a_list)

In [130]:
out1

,0,1,2,3,4
0,-84.425270,39.151750,"3228 Berwyn Pl, CINCINNATI, OH, 45209",2017,005301
1,-84.474525,39.130585,"1717 Fernwood St, CINCINNATI, OH, 45206",2009,004100
2,-84.439354,39.145546,"3731 Woodland Ave, CINCINNATI, OH, 45209",3007,005200
3,-84.291790,39.260227,"838 Oak Canyon Dr, LOVELAND, OH, 45140",1004,024322
4,-84.431046,39.130985,"3018 Kinmont St, CINCINNATI, OH, 45208",2007,004900
...,...,...,...,...,...
479,-84.487890,39.122055,"1023 Windsor St, CINCINNATI, OH, 45206",2006,002000
480,-84.554280,39.167450,"4510 Colerain Ave, CINCINNATI, OH, 45223",2000,007900
481,-84.587380,39.138622,"2615 Fleetwood Ave, CINCINNATI, OH, 45211",1000,010003
482,-84.470660,39.236897,"22 Chestnut Ave, CINCINNATI, OH, 45215",1005,022602


In [131]:
#Export data as csv
out1.to_csv('geocoded.csv',index=False)